### OBTENCION DE DATOS API MERCADOLIBRE

#### Importacion de librerias necesarias

In [1]:
import json
import requests
from urllib.parse import quote
import pandas as pd
from Publication import CarPublicationBuilder

#### Obtencion de publicaciones

El objetivo de esta seccion es obtener un listado de publicaciones de "vehiculos" desde la API de **MercadoLibre (https://developers.mercadolibre.com.ar/)** utilizando la libreria **requests** de python.<br>En primer lugar se crea una lista(**cars**) que aloja diferentes modelos de vehiculos disponibles en el mercado automotriz, posteriormente mediante el uso de la libreria **urllib** y su metodo **quote** se codifica la lista **cars** a fin de transformar ciertos caracteres para mantener la integridad de la URL al momento de realizar la request.

In [2]:
#43 autos
cars = [    'Volkswagen gol','Volkswagen gol trend', 'Volkswagen up', 'Volkswagen bora', 'Volkswagen vento', 'Volkswagen polo', 'Volkswagen amarok', 'Volkswagen golf',
            'Chevrolet corsa', 'Chevrolet onix', 'Chevrolet agile', 'Chevrolet prisma', 'Chevrolet cruze', 'Chevrolet aveo', 'Chevrolet tracker',
            'Renault clio', 'Renault sandero', 'Renault fluence', 'Renault logan', 'Reanault partner', 'Renault capture',
            'Peugeot 206', 'Peugeot 207', 'Peugeot 208', 'Peugeot 307','Peugeot 308', 'Peugeot 407', 'Peugeot 408',
            'Fiat cronos', 'Fiat argos', 'Fiat toro', 'Fiat palio', 'Fiat siena', 'Fiat uno',                   
            'Ford fiesta', 'Ford Ecosport', 'Ford ka', 'Ford ranger', 'Ford focus',
            'Toyota etios', 'Toyota hilux', 'Toyota corolla', 'Toyota yaris', 
            'Honda civic', 'Honda city', 'Honda cr-v']

#Se codifican los nombres
cars_codif = []
for car in cars:
    cars_codif.append(quote(car))

Con la finalidad de realizar una request por cada modelo de automovil en la lista, se crea la funcion **get_publication()**. Esta funcion  devuelve una lista de diccionarios, cada uno de estos diccionarios representa una publicacion de **MercadoLibre**. Dentro de la logica de la funcion se establece un maximo de 250 publicaciones por vehiculo.

In [10]:
def get_publications(car : str) -> list:
    publication = []
    url = f'https://api.mercadolibre.com/sites/MLA/search?q={car}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        results = data.get('paging', dict).get('total', None)  #TOTAL RESULTADOS EXISTENTES
        offset = 0 #POSICION DE INICIO PARA LA REQUEST

        if 'results' in data:
            for item in data['results']:
                publication.append(item)

            while (results > offset) and (results > 50):
                offset +=50 
                if offset == 350: break 
                url = f'https://api.mercadolibre.com/sites/MLA/search?q={car}&offset={offset}'
                response = requests.get(url)
                data = response.json()
                if 'results' in data:
                    for item in data['results']:
                        publication.append(item)  

        return publication                    
    else:
        return publication

Se crea una la lista **publications_list** la cual alojará los diccionarios obtenidos de la funcion **get_publication()**. Acto seguido ejecuta un ciclo for a fines de recorrer la lista **cars_codif**, ejecutar la funcion **get_publication()** por cada uno de los elementos en la lista, luego los resultados obtenidos son alojados en **publications_list**

In [11]:
#Lista para alojar las publicaciones
publications_list = []

for car in cars_codif:
    publications = get_publications(car)
    for publication in publications:
        publications_list.append(publication)

print(f'La cantidad de publicaciones obtenidas es: {len(publications_list)}')

La cantidad de publicaciones obtenidas es: 12196


Se guarda un archivo **JSON** con el resultado obtenido en el paso anterior

In [12]:
#GUARDADO DE JSON
with open('MeliDataPubl.json','w') as file:
    json.dump(publications_list, file, indent=4, separators=(',',':'))

#### Extraccion de los datos de interes

Con la finalidad de estructurar los datos extraidos de la API anteriormente se crea la clase **CarPublicaction()** la cual aloja en sus atributos los datos de interes de las publicaciones que se obtubieron. Tambien se crea la clase **CarPublicationBuilder()** cuya responsabilidad es extraer de un diccionario(publicacion) los datos de interes y devolver una instancia de la clase **CarPublicaction()** con sus atributos seteados

Mediante un ciclo for se recorre **publications_list** con la finalidad de ejecutar una instancia de la clase **CarPublicationBuilder()** y obtener una objeto de tipo **CarPublicaction()** por cada diccionario en **publications_list** 

In [16]:
car_publications = []

for item in publications_list:
    publication = CarPublicationBuilder(item).build()
    car_publications.append(publication)

print(f'Cantidad de registros obtenidos: {len(car_publications)}')

Cantidad de registros obtenidos: 12196


Se guarda el resultado obtenido en un archivo **JSON** de manera local

In [17]:
with open('MeliCarPublicactions.json','w' ) as file:
    json.dump(car_publications, file, indent=4, separators=(',',':'))

Utilizando **Pandas** se puede leer la lista de diccionarios (**car_publications**) para obtener un **Dataframe** y manipular los datos segun la necesidad.

In [18]:
df = pd.DataFrame(car_publications)
df.head(3)

,id,category_id,title,condition,car_year,brand,model,version,engine,engine_power,...,transmission,currency,price,seller_id,seller_nickname,is_car_shop,seller_country,seller_state,seller_city,seller_neighborhood
0,MLA1413685801,MLA1744,Volkswagen Gol 1.4 3ptas Año 2013 - Liv Motors,Usado,2013,Volkswagen,Gol,1.4 Power 83cv 3 p,1.4,83 hp,...,Manual,ARS,6000000,780050555,LIVMOTORS 2,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Munro
1,MLA1406864357,MLA1744,Volkswagen Gol 2012 1.4 Power Ps+ac 83cv,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,USD,10500,210095605,MASSEYGROUP EXCLUSIVOS,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Olivos
2,MLA1677544030,MLA1744,Volkswagen Gol Power 3p 2012 Financiación Con Dni,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,ARS,5800000,5571520,DASAUTOS2014,False,Argentina,Bs.As. G.B.A. Sur,Berazategui,Berazategui


Tambien se puede leer el archivo **JSON** creado anteriormente utilizando un **DataFrame** de **Pandas**

In [19]:
df = pd.read_json('MeliCarPublications.json')
df.head(3)

,id,category_id,title,condition,car_year,brand,model,version,engine,engine_power,...,transmission,currency,price,seller_id,seller_nickname,is_car_shop,seller_country,seller_state,seller_city,seller_neighborhood
0,MLA1413685801,MLA1744,Volkswagen Gol 1.4 3ptas Año 2013 - Liv Motors,Usado,2013,Volkswagen,Gol,1.4 Power 83cv 3 p,1.4,83 hp,...,Manual,ARS,6000000,780050555,LIVMOTORS 2,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Munro
1,MLA1406864357,MLA1744,Volkswagen Gol 2012 1.4 Power Ps+ac 83cv,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,USD,10500,210095605,MASSEYGROUP EXCLUSIVOS,False,Argentina,Bs.As. G.B.A. Norte,Vicente López,Olivos
2,MLA1677544030,MLA1744,Volkswagen Gol Power 3p 2012 Financiación Con Dni,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,ARS,5800000,5571520,DASAUTOS2014,False,Argentina,Bs.As. G.B.A. Sur,Berazategui,Berazategui


In [20]:
print(f'Cantidad de filas y columnas: {df.shape}\n\n')

df.info()

Cantidad de filas y columnas: (12196, 25)


<class 'pandas.core.frame.DataFrame'>
Index: 12196 entries, 0 to 12195
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   12196 non-null  object 
 1   category_id          12196 non-null  object 
 2   title                12196 non-null  object 
 3   condition            12196 non-null  object 
 4   car_year             12196 non-null  int64  
 5   brand                12196 non-null  object 
 6   model                12196 non-null  object 
 7   version              12196 non-null  object 
 8   engine               11885 non-null  object 
 9   engine_power         11349 non-null  object 
 10  doors                12196 non-null  int64  
 11  km                   12196 non-null  object 
 12  fuel_type            12196 non-null  object 
 13  traction_control     11809 non-null  object 
 14  passenger_capacity   11275 non-null  float64
 1